In [30]:
import os
from IPython.display import Image, display, clear_output
from astroquery.cadc import Cadc
import pandas as pd
import glue_jupyter as gj
import ipywidgets as widgets
from ipywidgets import interact
from cadcutils import net
from cadctap import CadcTapClient


observation_id = widgets.IntText(value=1013372,
                                 description='Observation ID',
                                 continuous_update=False,
                                 style={'description_width': 'initial'})
ui_database = widgets.HBox([observation_id])


def retrieve_data(observation_id):

    obs_query = """SELECT observationID FROM caom2.Plane AS Plane 
    JOIN caom2.Observation AS Observation 
    ON Plane.obsID = Observation.obsID
    WHERE (Observation.observationID = '{obs_id}'
    AND collection = '{collection}'
    AND instrument_name = '{instrument}')"""
    
    obs_query_param = {'obs_id': observation_id,
                       'collection': 'CFHT',
                       'instrument': 'MegaPrime'}
    
    cadc = Cadc()
    output = cadc.exec_sync(obs_query.format(**obs_query_param), output_file = None) #need a byte like object file 
    print(output)
    
    
    quality_query = """SELECT * FROM ml"""
    
    
    
    
database_output = widgets.interactive_output(retrieve_data,
                                             {"observation_id": observation_id})
display(ui_database , database_output)
    
 #   query = """SELECT TOP 1000 * FROM ml.Image_Quality_Table
 #   WHERE Good_Probability >= {good_probability}
 #   AND Number_of_Dead_CCDs != {invalid_number_of_dead_ccds}
 #   """
 #   query_param = {'good_probability': 0.2,
 #                  'invalid_number_of_dead_ccds': -99}
 #   
 #   certSubject = net.Subject(
 #       certificate=os.path.join(os.environ['HOME'], ".ssl/cadcproxy.pem"))
 #   
 #   client = CadcTapClient(certSubject)
 #   client.query(query.format(**query_param), output_file='tmp/result.xml', timeout=10)
 #   cadc = Cadc()
 #   cadc_query = """"""
 #   output = cadc.exec_sync(cadc_query, uploads = None, output_file = None)







Output()